In [1]:
import numpy as np, healpy as hp, pickle, prjlib, curvedsky, tqdm

In [2]:
ntype = 'base'
types = ['id','la','co','LA']
num = 50
p = {}
p['id'] = prjlib.analysis_init(t='id',ntype='cv')
p['LA'] = prjlib.analysis_init(t='la',freq='com',fltr='none',ntype=ntype+'_roll50')
p['la'] = prjlib.analysis_init(t='la',freq='com',fltr='cinv',ntype=ntype+'_roll50')
p['co'] = prjlib.analysis_init(t='co',freq='com',fltr='cinv',ntype=ntype+'_roll50')

In [3]:
lmax, l = {}, {}
for c in types: lmax[c] = 4096
lmax['co'] = 2048
for c in types: l[c] = np.linspace(0,lmax[c],lmax[c]+1)

In [ ]:
Wsa = hp.pixelfunc.ud_grade(prjlib.window('sa')[0],2048)
Wla = prjlib.window('la',ascale=0)[0] # here, apodized mask is used otherwise the correlations at edge gets worse
Wco = Wsa * Wla #hp.pixelfunc.ud_grade(Wla,512)

The loaded SAT mask is further multiplied by hit-count "binary" (not hit count itself)


/global/homes/t/toshiyan/.local/lib/python3.7/site-packages/healpy/fitsfunc.py:391: UserWarning: NSIDE = 512
  warnings.warn("NSIDE = {0:d}".format(nside))
/global/homes/t/toshiyan/.local/lib/python3.7/site-packages/healpy/fitsfunc.py:400: UserWarning: ORDERING = RING in fits file
  warnings.warn("ORDERING = {0:s} in fits file".format(ordering))
/global/homes/t/toshiyan/.local/lib/python3.7/site-packages/healpy/fitsfunc.py:428: UserWarning: INDXSCHM = IMPLICIT
  warnings.warn("INDXSCHM = {0:s}".format(schm))


In [5]:
Elm, Blm = {}, {}
EE, BB, xE, xB = {}, {}, {}, {}
rhoE, rhoB = {}, {} 
for c in ['id','LA','la','co']:
    rhoE[c], rhoB[c] = 0., 0.

In [6]:
for i in tqdm.tqdm(range(1,num+1)):
    for c in ['id','LA','la']:
        Elm[c] = pickle.load(open(p[c].fcmb.alms['o']['E'][i],"rb"))[:lmax[c]+1,:lmax[c]+1]
        Blm[c] = pickle.load(open(p[c].fcmb.alms['o']['B'][i],"rb"))[:lmax[c]+1,:lmax[c]+1]
        # cutsky for E
        Elm[c] = curvedsky.utils.mulwin_spin(Elm[c],0*Elm[c],Wla)[0]
        Blm[c] = curvedsky.utils.mulwin_spin(0*Blm[c],Blm[c],Wla)[1]
        # cl
        EE[c,i] = curvedsky.utils.alm2cl(lmax[c],Elm[c])
        BB[c,i] = curvedsky.utils.alm2cl(lmax[c],Blm[c])
        if c=='id': continue
        xE[c,i] = curvedsky.utils.alm2cl(lmax[c],Elm[c],Elm['id'][:lmax[c]+1,:lmax[c]+1])
        xB[c,i] = curvedsky.utils.alm2cl(lmax[c],Blm[c],Blm['id'][:lmax[c]+1,:lmax[c]+1])
        # corr coeff
        rhoE[c] += xE[c,i]**2/EE[c,i]/EE['id',i][:lmax[c]+1]/num
        rhoB[c] += xB[c,i]**2/BB[c,i]/BB['id',i][:lmax[c]+1]/num

  0%|          | 0/50 [00:00<?, ?it/s]/global/homes/t/toshiyan/.conda/envs/loc/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/global/homes/t/toshiyan/.conda/envs/loc/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
100%|██████████| 50/50 [8:36:43<00:00, 620.08s/it]  


In [7]:
for c in ['id','co']:
    rhoE[c], rhoB[c] = 0., 0.
for i in tqdm.tqdm(range(1,num+1)):
    for c in ['id','co']:
        Elm[c] = pickle.load(open(p[c].fcmb.alms['o']['E'][i],"rb"))[:lmax[c]+1,:lmax[c]+1]
        Blm[c] = pickle.load(open(p[c].fcmb.alms['o']['B'][i],"rb"))[:lmax[c]+1,:lmax[c]+1]
        # cutsky for E
        Elm[c] = curvedsky.utils.mulwin_spin(Elm[c],0*Elm[c],Wco)[0]
        Blm[c] = curvedsky.utils.mulwin_spin(0*Blm[c],Blm[c],Wco)[1]
        # cl
        EE[c,i] = curvedsky.utils.alm2cl(lmax[c],Elm[c])
        BB[c,i] = curvedsky.utils.alm2cl(lmax[c],Blm[c])
        if c=='id': continue
        xE[c,i] = curvedsky.utils.alm2cl(lmax[c],Elm[c],Elm['id'][:lmax[c]+1,:lmax[c]+1])
        xB[c,i] = curvedsky.utils.alm2cl(lmax[c],Blm[c],Blm['id'][:lmax[c]+1,:lmax[c]+1])
        # corr coeff
        rhoE[c] += xE[c,i]**2/EE[c,i]/EE['id',i][:lmax[c]+1]/num
        rhoB[c] += xB[c,i]**2/BB[c,i]/BB['id',i][:lmax[c]+1]/num
    #rhoE[c] /= num
    #rhoB[c] /= num

  0%|          | 0/50 [00:00<?, ?it/s]/global/homes/t/toshiyan/.conda/envs/loc/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide
/global/homes/t/toshiyan/.conda/envs/loc/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
100%|██████████| 50/50 [3:44:24<00:00, 269.29s/it]  


In [8]:
for c in types:
    if c=='id': continue
    np.savetxt('../../data/sodelens/paper/2021_pipeline/cinv_rho_'+c+'.dat',np.array((rhoE[c],rhoB[c])).T)